In [ ]:
# Script programering 6-22

# pip https://code.google.com/archive/p/lorem-ipsum-generator/

import tempfile
from lorem import LoremGenerator

with tempfile.SpooledTemporaryFile(mode="w+t", max_size=10240) as tmp:
    paragraph = LoremGenerator().gen_paragraph(
        comma=(0, 2),           
        word_range=(4, 8),     
        sentence_range=(3, 7)   
    )
    tmp.write(paragraph)
    tmp.seek(0)
    print(tmp.read())
    tmp.seek(0)
    temp = tmp.read()
    temp = temp.replace("ipsum", "dolor")
    temp = temp.replace("elit", "dolor")
    temp = temp.replace("adipiscing", "dolor")
    temp = temp.replace("commodo", "dolor")
    tmp.seek(0)
    tmp.truncate()
    tmp.write(temp)
    tmp.seek(0)
    print(tmp.read())
    
    
    

In [ ]:
# Script programering 6-23

import tempfile

with tempfile.SpooledTemporaryFile(mode="w+t", max_size=10240) as tmp:
    csv = f'''"result","tongue","decide","sugar","express","mighty"
"put","frequently","hardly","brother","maybe","baby"
"perfect","height","opinion","strong","bite","slabs"
"alphabet","slipped","total","rock","studied","pen"
"bound","region","decide","continent","seldom","important"
"rapidly","vote","cause","told","immediately","last"
"spider","average","vertical","paint","loose","poor"
    '''
    tmp.write(csv)
    tmp.seek(0)
    print(tmp.read())
    tmp.seek(0)
    
    with tempfile.SpooledTemporaryFile(mode="w+t", max_size=10240) as newtmp:
        for x in tmp.readlines():
            if newtmp.tell() > 0:
                newtmp.seek(newtmp.tell() - 1)
                newtmp.write("\n")
            for y in x.split(","):
                y = y.replace('"', '').strip()
                if len(y) > 5 and ("a" in y or "e" in y or "d" in y):
                    newtmp.write('"{}",'.format(y))
                else:
                    newtmp.write('"",')
        newtmp.seek(0)
        print(newtmp.read())

In [ ]:
# Script programering 6-24

import zipfile
import os

file_list = []
for file in os.listdir("."):
    if os.path.isfile(file):
        file_list.append(file)
        
with zipfile.ZipFile("zipped.zip", mode="w") as archive:
    for x in file_list:
        archive.write(x)

In [ ]:
# Script programering 6-25

import shutil
shutil.copytree("./", "./backup/")

In [ ]:
# Script programering 6-26

# Ingen garanti att detta fungerar på windows

import os 
import shutil
import datetime
import json

if os.name == "nt":
    safepath = "%documents%/backup"
    #Windows (¬_¬)
else:
    safepath = os.path.expanduser("~/backups")
    # Linux ദ്ദി(｡•̀ ,<)~✩‧₊
while True:
    try:
        whatdo = bool(int(input("Do you want to create a backup[0] or restore from a backup[1]? [0/1]: ")))
        break
    except ValueError:
        print("Please enter 0 or 1.")
        
if whatdo == 0:
    while True:
        path = str(input("What folder do you want to back up? (leave blank for current folder): "))
        if path == "":
            path = os.path.abspath(".")
        if os.path.exists(path) and os.path.isdir(path):
            break
        else:
            print("Folder does not exist.")

    folder_name = os.path.basename(os.path.normpath(path))
    backup_folder = os.path.join(
        safepath,
        f"{folder_name}-" + datetime.datetime.now().strftime("%Y.%m.%d-%H.%M.%S")
    )
    shutil.copytree(path, backup_folder)
    
    # create backup.json
    os.makedirs(safepath, exist_ok=True)
    if not os.path.exists("{}/backups.json".format(safepath)):
        open("{}/backups.json".format(safepath), "w").close()
    
    # open backup.json
    with open("{}/backups.json".format(safepath), "r+") as f:
        if os.stat("{}/backups.json".format(safepath)).st_size == 0:
            f.write("{}")
            f.seek(0)
   
    # write to backup.json
    with open("{}/backups.json".format(safepath), "r+") as f:
        data = json.load(f)
        if folder_name not in data:
            data[folder_name] = []
        data[folder_name].append({
            "path": backup_folder,
            "original_path": path,
            "date": datetime.datetime.now().strftime("%Y.%m.%d-%H.%M.%S")
        })
        f.seek(0)
        json.dump(data, f, indent=4)
        f.truncate()
        f.close()
    print("{} backed up successfully to {}".format(path,backup_folder))
elif whatdo == 1:
    # save json to array
    with open("{}/backups.json".format(safepath), "r") as f:
        data = json.load(f)
        
    # check for deleted backups
    changed = False
    for folder in data:
        for backup in data[folder]:
            if not os.path.exists(backup["path"]):
                data[folder].remove(backup)
                changed = True
    if changed == True:
        with open("{}/backups.json".format(safepath), "w") as f:
            json.dump(data, f, indent=4)
            f.truncate()
            f.close()
            
    # show options
    print(data)
    y = 0
    for x in data:
        print(f"\nBackups for folder: {x} : [{y}]")
        y += 1
        for i, backup_entry in enumerate(data[x]):
            print(f"[{i}] {backup_entry['date']} - {backup_entry['original_path']}")
            
    while True: 
        try:
            foldback = int(input("Choose a folder to backup by entering a number: "))
            if foldback < 0 or foldback >= len(data):
                print("Please enter a valid number.")
                continue
            break
        except ValueError:
            print("Please enter a number")
    folder_key = list(data)[foldback]
    print("Restoring backup for {}".format(data[folder_key][0]["original_path"]))
    num_versions = len(data[folder_key])
    while True:
        try:
            versback = int(input(f"Choose a version to restore (0-{num_versions-1}): "))
            if versback < 0 or versback >= num_versions:
                print("Please enter a valid version number.")
                continue
            break
        except ValueError:
            print("Please enter a number")
    
    #restore backup
    backup_path = data[folder_key][versback]["path"]
    original_path = data[folder_key][versback]["original_path"]
    if os.path.exists(original_path):
        shutil.rmtree(original_path)
    shutil.copytree(backup_path, original_path)
    print("Backup from {} restored to {}".format(backup_path, original_path))
    

In [ ]:
# Script programering 6-27

import tempfile
import datetime
import os

if not os.path.exists("data.json") or os.stat("data.json").st_size == 0:
    with tempfile.SpooledTemporaryFile(mode="w+t", max_size=10240) as tmp:
        dateandtime = {"creation_timestamp": datetime.datetime.now().strftime("%Y.%m.%d-%H.%M.%S")}
        import json
        data = {
            "apple" : 1.50,
            "banana" : 1.98,
            "pineapple" : 2.97
            }
        json.dump((dateandtime, data), tmp)
        tmp.seek(0)
    
        with open("data.json", "w") as f:
            f.write(tmp.read())

f = open("data.json", "r")
print(f.read())
    

In [ ]:
# Script programering 6-28

import tempfile

with tempfile.SpooledTemporaryFile(mode="w+t", max_size=10240) as tmpcsv:
    tmpcsv.write("Timestamp,Location,Temperature_C\n2025-09-10 08:00,Greenhouse,22.5\n2025-09-10 08:05,Greenhouse,22.7\n2025-09-10 08:10,Greenhouse,23.1\n2025-09-10 08:15,Greenhouse,22.8\n2025-09-10 08:20,Greenhouse,23.3\n2025-09-10 08:25,Greenhouse,23.0\n2025-09-10 08:30,Greenhouse,23.6\n2025-09-10 08:35,Greenhouse,23.8\n2025-09-10 08:40,Greenhouse,24.1\n2025-09-10 08:45,Greenhouse,23.9\n2025-09-10 08:50,Outdoor,-1.2\n2025-09-10 08:55,Outdoor,-0.8\n2025-09-10 09:00,Outdoor,-2.3\n2025-09-10 09:05,Outdoor,-1.5\n2025-09-10 09:10,Outdoor,0.4\n2025-09-10 09:15,Outdoor,-3.1\n2025-09-10 09:20,Outdoor,-2.7\n2025-09-10 09:25,Outdoor,-1.9\n2025-09-10 09:30,Outdoor,0.2\n2025-09-10 09:35,Outdoor,-0.5\n2025-09-10 09:40,Cellar,5.3\n2025-09-10 09:45,Cellar,5.5\n2025-09-10 09:50,Cellar,5.7\n2025-09-10 09:55,Cellar,5.8\n2025-09-10 10:00,Cellar,6.1\n2025-09-10 10:05,Cellar,6.0\n2025-09-10 10:10,Cellar,5.9\n2025-09-10 10:15,Cellar,6.3\n2025-09-10 10:20,Cellar,6.1\n2025-09-10 10:25,Cellar,6.2")
    tmpcsv.seek(0)
    with tempfile.SpooledTemporaryFile(mode="w+t", max_size=10240) as newcsv:
        tmpcsv.seek(0)
        lines = tmpcsv.read().split("\n")
        for x in lines[1:]: 
            temperature = float((x.split(",")[2]))
            if temperature > 0:
                newcsv.write("{}\n".format(temperature))
        newcsv.seek(0)
        print(newcsv.read())

In [ ]:
# Script programering 6-29

import platform, subprocess
from datetime import datetime, timedelta

def getlogs():
    now = datetime.now()
    minutes = int(input("How many minutes back do you want the logs to go, enter in minutes: "))
    since = now - timedelta(minutes)

    if platform.system() == "Linux":
        # Linux ദ്ദി(｡•̀ ,<)~✩‧₊
        try:
            return subprocess.check_output(
                ["journalctl", "--since", since.strftime('%Y-%m-%d %H:%M:%S')],
                text=True
            )
        except:
            try:
                with open("/var/log/syslog") as f:
                    return "".join(line for line in f if since.strftime('%b %d %H:') in line)
            except:
                return "Cannot read logs"
    elif platform.system()== "Windows":
        #Windows (¬_¬)
        ps = f"$s=(Get-Date).AddMinutes(-{minutes}); Get-WinEvent -FilterHashtable @{{LogName='System'; StartTime=$s}} | Format-List TimeCreated, Id, LevelDisplayName, Message"
        return subprocess.check_output(["powershell", "-Command", ps], text=True)
    else:
        print("Unsupported OS")
        return "Unsupported OS"

print(getlogs())


In [ ]:
# Script programering 6-30

import tempfile
import struct
import random
import numpy

numbers = []

for x in range(0,1000):
    numbers.append(random.randint(0,99))
    
with tempfile.SpooledTemporaryFile(mode="w+b", max_size=10240) as tmp:
    for x in numbers:
        tmp.write(struct.pack("I", x))
    tmp.seek(0)
    
    extractednums = []
    
    while bytes_read := tmp.read(4):
        extractednums.append(struct.unpack('<I', bytes_read)[0])
        
    sum = 0
    for x in extractednums:
        sum += x
    
    mean = sum/len(extractednums)
    
    SD = numpy.std(extractednums)
    
    print("The mean is {}".format(mean))
    print("The standard deviation is {}".format(SD))
    


In [ ]:
# Script programering 6-31

In [ ]:
# Script programering 6-32

In [ ]:
# Script programering 6-33

In [ ]:
# Script programering 6-34

In [ ]:
# Script programering 6-35